# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


/Users/silverwhale/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
df_raw = pd.read_csv("admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [15]:
# frequency table for prestige and whether or not someone was admitted
print df.groupby('prestige').count()

          admit  gre  gpa
prestige                 
1.0          61   61   61
2.0         148  148  148
3.0         121  121  121
4.0          67   67   67


In [16]:
print df.groupby('admit').count()

       gre  gpa  prestige
admit                    
0      271  271       271
1      126  126       126


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [68]:
df_prestige = pd.get_dummies(df['prestige'], prefix="prestige")
print df_prestige.head()
df_new = df.join(df_prestige)
print df_new.head()

   prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0             0             0             1             0
1             0             0             1             0
2             1             0             0             0
3             0             0             0             1
4             0             0             0             1
   admit    gre   gpa  prestige  prestige_1.0  prestige_2.0  prestige_3.0  \
0      0  380.0  3.61       3.0             0             0             1   
1      1  660.0  3.67       3.0             0             0             1   
2      1  800.0  4.00       1.0             1             0             0   
3      1  640.0  3.19       4.0             0             0             0   
4      0  520.0  2.93       4.0             0             0             0   

   prestige_4.0  
0             0  
1             0  
2             0  
3             1  
4             1  


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We only need n-1 class variables since the n-th variable is all n-1 variables with value of 0.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [51]:
#NOTE: Getting error with this code so commented it out
#cols_to_keep = ['admit', 'gre', 'gpa']
#handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
#print handCalc.head()

In [69]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df_new.prestige, df_new.admit, margins=True)

admit,0,1,All
prestige,,,
1.0,28,33,61
2.0,95,53,148
3.0,93,28,121
4.0,55,12,67
All,271,126,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [70]:
pd.crosstab(df_new['prestige_1.0'], df_new.admit, margins=True)

admit,0,1,All
prestige_1.0,,,
0,243,93,336
1,28,33,61
All,271,126,397


In [60]:
(33.0/28.0)

1.1785714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [62]:
(93.0/243.0)

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [63]:
(33.0 / 28.0) / (93.0 / 243.0)

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: You are more than 3 times more likely of being admitted to a top grad school if you attended a #1 ranked college than if you did not.

#### 3.5 Print the cross tab for prestige_4

In [72]:
pd.crosstab(df_new['prestige_4.0'], df_new.admit, margins=True)

admit,0,1,All
prestige_4.0,,,
0,216,114,330
1,55,12,67
All,271,126,397


#### 3.6 Calculate the OR 

In [75]:
#Odds of being admitted to a top grad school if you attended a #4 ranked college than if you did not.
P4_a=(12.0/55.0)
P4_na=(114.0/216.0)
P4_OR=(P4_a/P4_na)
print P4_OR

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: You are more less than half as likely of being admitted to a top grad school if you attended a #4 ranked college than if you did not.

## Part 4. Analysis

In [76]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(df_prestige.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


/Users/silverwhale/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [77]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [78]:
  train_cols = data.columns[1:]

#### 4.2 Fit the model

In [80]:
# Statsmodels logistic regression is sm.Logit
import statsmodels.api as sm


logit = sm.Logit(data['admit'], data[train_cols])

result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [81]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 25 Sep 2017   Pseudo R-squ.:                 0.08166
Time:                        10:42:15   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [83]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [84]:
  params = result.params
  conf = result.conf_int()
  conf['OR'] = params
  conf.columns = ['2.5%', '97.5%', 'OR']
  print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: You are half as likely of being admitted to a top grad school if you attended a #2 ranked college than if you attended a #1 ranked college.

#### 4.6 Interpret the OR of GPA

Answer: Answer: You are more than 2 times as likely of being admitted to a top grad school with a higher GPA than with a lower GPA.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [85]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [93]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [94]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
combos_dummy = pd.get_dummies(combos['prestige'], prefix='prestige')
combos_dummy.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(combos_dummy.ix[:, 'prestige_2':])
combos.head()

/Users/silverwhale/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  import sys


,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1.0,1.0,0,0,0
1,220.0,2.260000,2.0,1.0,1,0,0
2,220.0,2.260000,3.0,1.0,0,1,0
3,220.0,2.260000,4.0,1.0,0,0,1
4,220.0,2.453333,1.0,1.0,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [98]:
combos['admit_prediction'] = result.predict(combos[['gre','gpa','intercept','prestige_2','prestige_3','prestige_4']])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4,admit_prediction
0,220.0,2.260000,1.0,1.0,0,0,0,0.827672
1,220.0,2.260000,2.0,1.0,1,0,0,0.557380
2,220.0,2.260000,3.0,1.0,0,1,0,0.503951
3,220.0,2.260000,4.0,1.0,0,0,1,0.090492
4,220.0,2.453333,1.0,1.0,0,0,0,0.848114


#### 5.3 Interpret findings for the last 4 observations

In [99]:
combos.tail(4)

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4,admit_prediction
396,800.0,4.0,1.0,1.0,0,0,0,0.985399
397,800.0,4.0,2.0,1.0,1,0,0,0.946509
398,800.0,4.0,3.0,1.0,0,1,0,0.934534
399,800.0,4.0,4.0,1.0,0,0,1,0.582995


Answer: Students with the highest GPA and GRE scores from schools ranked 1,2,3 have a greater than 90% likelihood of being admitted to a top graduate school, whereas students with the same charactersistics from a school ranked 4 has less than 60% change of admission.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.